In [ ]:
#expanding abbrevations in a query -> combine with question query if there is a question query which contains an abbrevation
#test phrase based expansion, expand to "This paper introduces Information Retrieval" -> set retrieval, boolean retrieval?
#adjust bm25 parameters
#adjust question prompt so that it catches the intent behind why questions better
#What and Why are probably most common question words -> specialize on them

Thesis

Setup

In [12]:
# Imports
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt
import pandas as pd
import re
from collections import Counter

In [2]:
# Create a REST client to the TIRA platform for retrieving the pre-indexed data.
ensure_pyterrier_is_loaded()
tira = Client()

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [3]:
# The dataset: the union of the IR Anthology and the ACL Anthology
# This line creates an IRDSDataset object and registers it under the name provided as an argument.
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

# A (pre-built) PyTerrier index loaded from TIRA
index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)

## Build Ngram Index

In [5]:
docs =  pt_dataset.get_corpus_iter()
docs = list(docs)
count = sum(1 for _ in docs)
docs = docs[:126959]
print("Number of documents:", count)

ir-lab-sose-2024/ir-acl-anthology-20240504-training documents: 100%|██████████| 126958/126958 [00:02<00:00, 57238.14it/s]

Number of documents: 126958


In [7]:
#Method that removes all special characters from a String, and returns either a String or a list of all words
def clean_text(text, return_as_list = False):
    text = text.replace('\n', ' ')
    cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', ' ', text) #remove non-alphanumeric characters, except spaces
    if return_as_list:
        word_list = cleaned_text.split()
        return word_list
    else:
        return cleaned_text

In [8]:
import nltk
from nltk.corpus import stopwords
import re

# Ensure NLTK data directory is set correctly
nltk.data.path.append("/usr/local/nltk_data")

# Download 'stopwords' corpus to the specified directory
nltk.download('stopwords', download_dir="/usr/nltk_data")

# Get the set of stopwords for the English language
stop_words = set(stopwords.words('english'))

def remove_stopwords(text, return_as_list = False):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    # Join the filtered words back into a single string

    if return_as_list:
        return filtered_words
    else:
        return ' '.join(filtered_words)

[nltk_data] Downloading package stopwords to /usr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer

stemmer = SnowballStemmer('english')

def stem_text(text, return_as_list = False):
    words = text.split()

    stemmed_words = [stemmer.stem(word) for word in words]
    if return_as_list:
        return stemmed_words
    # Join the stemmed words back into a single string
    else:
        return ' '.join(stemmed_words)

In [10]:
#This is our ngram tokenizer. It takes a string and returns a dict of all ngrams, where each ngram is seperated by $$ so it will be parsed as one token

def tokenize_ngrams_to_dict(text, n1=1, n2=2):
    
    # Tokenize the text into words
    words = text.split(' ')
    words = [word for word in words if len(''.join(format(ord(c), '08b') for c in word)) <= 60]

    # Initialize an empty Counter to hold all n-grams
    all_ngram_counts = Counter()
    
    # Loop through each n from n1 to n2
    for n in range(n1, n2 + 1):
        # Generate n-grams for the current n
        ngrams = ['$$'.join(words[i:i+n]) for i in range(len(words)-n+1)]
        
        # Update the Counter with the current n-grams
        all_ngram_counts.update(ngrams)
    
    return dict(all_ngram_counts)

In [13]:
for doc in docs:
        if 'text' in doc:
            doc['text'] = clean_text(doc['text'])
            doc['text'] = remove_stopwords(doc['text'])
            doc['text'] = stem_text(doc['text'])
            
            doc_1gram = tokenize_ngrams_to_dict(doc['text'], n1=1, n2=2) # Apply n-gram tokenization to the dataset

            doc['toks'] = doc_1gram # create new toks field for tokenfrequency
            del doc['text']  #This will delete the 'text' field from the documents
    
for i, doc in enumerate(docs):
     if i == 3:
           break
     print(doc)

#remove all empty documents
docs = [d for d in docs if any(k != '' for k in d['toks'].keys())]

{'docno': 'O02-2002', 'toks': {'studi': 1, 'word': 7, 'similar': 8, 'use': 3, 'context': 5, 'vector': 3, 'model': 2, 'need': 1, 'measur': 2, 'process': 1, 'natur': 1, 'languag': 1, 'especi': 1, 'general': 1, 'classif': 1, 'exampl': 1, 'base': 3, 'usual': 1, 'two': 1, 'defin': 1, 'accord': 3, 'distanc': 1, 'semant': 6, 'class': 2, 'less': 1, 'consid': 1, 'syntact': 5, 'ie': 2, 'howev': 1, 'real': 1, 'applic': 1, 'requir': 1, 'weight': 1, 'differ': 1, 'mixtur': 1, 'paper': 1, 'propo': 1, 'relat': 1, 'co': 2, 'occurr': 2, 'adopt': 1, 'inform': 1, 'theoret': 1, 'solv': 1, 'problem': 1, 'data': 1, 'spar': 1, 'preci': 1, 'featur': 2, 'deriv': 1, 'par': 1, 'environ': 1, 'adjust': 1, 'idf': 1, 'inver': 1, 'valu': 2, 'agglom': 1, 'cluster': 1, 'appli': 1, 'group': 2, 'turn': 1, 'togeth': 1, 'studi$$word': 1, 'word$$similar': 4, 'similar$$use': 1, 'use$$context': 1, 'context$$vector': 3, 'vector$$model': 1, 'model$$need': 1, 'need$$measur': 1, 'measur$$word': 1, 'similar$$process': 1, 'process$$

In [15]:
# Initialize the IterDictIndexer with pretokenised set to True
iter_indexer = pt.IterDictIndexer("./ngramindex", overwrite=True, meta={'docno': 35}, pretokenised=True, verbose = True, type = pt.index.IndexingType.SINGLEPASS)

# Index our pretokenized documents
index_ref = iter_indexer.index(docs)

index_ngram = pt.IndexFactory.of(index_ref)

#Print some stats about our index
print(index_ngram.getCollectionStatistics())

# Access the MetaIndex and Lexicon
meta = index_ngram.getMetaIndex()
lexicon = index_ngram.getLexicon()


i = 0
for term, le in index.getLexicon():
    i = i+1
    if i == 5:
        break
    print(term) 
    print(le.getFrequency())

Number of documents: 126824
Number of terms: 1735219
Number of postings: 11740755
Number of fields: 0
Number of tokens: 14688164
Field names: []
Positions:   false

0
9406
00
278
000
1761
0001
14


In [ ]:
#(add question query, why, to further see impact)

In [ ]:
#identify if query contains an abbrevation

In [ ]:
#identify if query is a question

In [ ]:
#expand selected query accoridingly -> play with tempereature
#prompt should react differently for What against Why questions
#question querys should be extracted from the dataframe
#querys that contain abbrevations but are not a question can remain 

In [ ]:
#(write prompt that uses phrase-based expansion)

In [ ]:
#rank querys without question querys with bm25
#maybe adjust bm25 parameters here

In [ ]:
#iterate over each subquery generated for a question query
#rank each subquery with bm25.search then aggregate the score for the question query

In [ ]:
#merge the resulting dataframe with the already done bm25 ranking dataframe

In [ ]:
#save and normalize run